In [3]:
import scanpy as sc
import scipy.io as sio
import numpy as np
import pandas as pd
import scipy.sparse # 导入scipy.sparse模块用于类型检查

dataset_name = 'hNSC'
input_filename = f'{dataset_name}_raw.h5ad'
output_filename = f'{dataset_name}_preprocessed.mat'
output_h5ad = f'{dataset_name}_preprocessed.h5ad'

print("--- 1. 加载数据 ---")
try:
    adata = sc.read_h5ad(input_filename)
    print(f"成功加载数据，原始维度: {adata.n_obs} 个细胞 x {adata.n_vars} 个基因")
    adata = adata[np.random.permutation(adata.n_obs)]
except FileNotFoundError:
    print(f"错误：未找到 '{input_filename}' 文件。请检查文件名和路径。")
    exit()


print("\n--- 2. 执行预处理与 dpFeature 基因筛选 ---")

# --- 对应 dpFeature 步骤 1: 基因过滤 ---
print("步骤 2.1: 过滤低表达基因 (在少于 5% 的细胞中表达的基因)")
min_cells_threshold = int(adata.n_obs * 0.05)
print(f"细胞总数: {adata.n_obs}。基因表达的最小细胞数阈值: {min_cells_threshold}")
sc.pp.filter_genes(adata, min_cells=min_cells_threshold)
print(f"过滤后维度: {adata.n_obs} 个细胞 x {adata.n_vars} 个基因")

# --- 论文中提到的对数转换 ---
print("\n步骤 2.2: 进行 log1p 对数转换")
sc.pp.log1p(adata)

# --- 对应 dpFeature 步骤 2 & 3: PCA 和降维空间聚类准备 ---
print("\n步骤 2.3: 运行 PCA 并构建细胞邻近图")

# 动态设置 n_comps 以避免错误。
n_samples = adata.n_obs
n_features = adata.n_vars
max_n_comps = min(n_samples, n_features) - 1
n_comps_to_use = min(50, max_n_comps)

print(f"数据集中有 {n_samples} 个细胞和 {n_features} 个基因。")
print(f"PCA 将计算 {n_comps_to_use} 个主成分。")

sc.pp.pca(adata, n_comps=n_comps_to_use)

sc.pp.neighbors(adata)      # 基于PCA空间构建邻近图

# --- 对应 dpFeature 步骤 4: 聚类 ---
print("\n步骤 2.4: 使用 Leiden 算法进行细胞聚类")
sc.tl.leiden(adata, resolution=0.5, key_added="dp_clusters")
print(f"聚类完成，共得到 {len(adata.obs['dp_clusters'].unique())} 个细胞簇")


# --- 对应 dpFeature 步骤 5 & 6: 差异表达分析和基因筛选 ---
print("\n步骤 2.5: 在簇之间进行差异表达基因分析")
sc.tl.rank_genes_groups(adata, groupby='dp_clusters', method='wilcoxon', n_genes=200)

print("\n步骤 2.6: 筛选前1000个差异最显著的基因作为特征")
deg_results = pd.DataFrame(adata.uns['rank_genes_groups']['names'])
top_genes = set()
for col in deg_results.columns:
    top_genes.update(deg_results[col].dropna())
    if len(top_genes) >= 1000:
        break
top_genes_list = list(top_genes)[:1000]

print(f"已成功筛选出 {len(top_genes_list)} 个特征基因。")

# --- 根据筛选出的基因，创建最终的数据对象 ---
print("\n--- 3. 创建并过滤最终的 AnnData 对象 ---")
adata_final = adata[:, top_genes_list].copy()
print(f"最终数据维度: {adata_final.n_obs} 个细胞 x {adata_final.n_vars} 个基因")


# --- 导出为 .mat 文件和 .h5ad 文件 ---
print("\n--- 4. 导出为 .mat 和 .h5ad 文件 ---")
# ====================================================
# 检查 adata_final.X 是稀疏矩阵还是密集矩阵，然后进行相应处理
if isinstance(adata_final.X, scipy.sparse.spmatrix):
    # 如果是稀疏矩阵，调用 .toarray()
    expression_matrix = adata_final.X.toarray()
else:
    # 如果已经是密集矩阵 (numpy.ndarray)，直接使用
    expression_matrix = adata_final.X

# 转置矩阵，使行为基因，列为细胞
expression_matrix = expression_matrix.T
# ====================================================

# 保存 .mat 文件
output_dict = {
    'expression_matrix': expression_matrix,
    'gene_names': adata_final.var_names.to_numpy(dtype=object),  # 基因名称作为行名
    'cell_names': adata_final.obs_names.to_numpy(dtype=object)   # 细胞名称作为列名
}

sio.savemat(output_filename, output_dict)

# 保存 .h5ad 文件
adata_final.write(output_h5ad)

print(f"\n处理完成！数据已保存为:")
print(f"1. MAT文件 '{output_filename}':")
print(f"  - 'expression_matrix': 一个 {expression_matrix.shape[0]}x{expression_matrix.shape[1]} 的 NumPy 数组 (基因 x 细胞)")
print(f"  - 'gene_names': 一个包含 {len(output_dict['gene_names'])} 个基因名称的数组")
print(f"  - 'cell_names': 一个包含 {len(output_dict['cell_names'])} 个细胞名称的数组")
print(f"2. H5AD文件 '{output_h5ad}'")

--- 1. 加载数据 ---
成功加载数据，原始维度: 2493 个细胞 x 32639 个基因

--- 2. 执行预处理与 dpFeature 基因筛选 ---
步骤 2.1: 过滤低表达基因 (在少于 5% 的细胞中表达的基因)
细胞总数: 2493。基因表达的最小细胞数阈值: 124


c:\Users\yLong\miniconda3\Lib\site-packages\scanpy\preprocessing\_simple.py:293: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["n_cells"] = number


过滤后维度: 2493 个细胞 x 7777 个基因

步骤 2.2: 进行 log1p 对数转换

步骤 2.3: 运行 PCA 并构建细胞邻近图
数据集中有 2493 个细胞和 7777 个基因。
PCA 将计算 50 个主成分。

步骤 2.4: 使用 Leiden 算法进行细胞聚类
聚类完成，共得到 7 个细胞簇

步骤 2.5: 在簇之间进行差异表达基因分析

步骤 2.6: 筛选前1000个差异最显著的基因作为特征
已成功筛选出 1000 个特征基因。

--- 3. 创建并过滤最终的 AnnData 对象 ---
最终数据维度: 2493 个细胞 x 1000 个基因

--- 4. 导出为 .mat 和 .h5ad 文件 ---

处理完成！数据已保存为:
1. MAT文件 'hNSC_preprocessed.mat':
  - 'expression_matrix': 一个 1000x2493 的 NumPy 数组 (基因 x 细胞)
  - 'gene_names': 一个包含 1000 个基因名称的数组
  - 'cell_names': 一个包含 2493 个细胞名称的数组
2. H5AD文件 'hNSC_preprocessed.h5ad'


In [4]:
adata.obs

,cell_cycle_phase,original_cell_id,cycle_phase,cell_phase,stage,true,dp_clusters
G1_cell45_count,G1,G1_cell45_count,G1,G1,0,G1,2
G2M_cell26_count,G2M,G2M_cell26_count,G2M,G2M,2,G2M,2
G1_cell4_count,G1,G1_cell4_count,G1,G1,0,G1,2
S_cell9_count,S,S_cell9_count,S,S,1,S,1
S_cell48_count,S,S_cell48_count,S,S,1,S,1
...,...,...,...,...,...,...,...
G2M_cell70_count,G2M,G2M_cell70_count,G2M,G2M,2,G2M,2
S_cell54_count,S,S_cell54_count,S,S,1,S,1
G2M_cell65_count,G2M,G2M_cell65_count,G2M,G2M,2,G2M,0
G2M_cell91_count,G2M,G2M_cell91_count,G2M,G2M,2,G2M,1
